## Dependencies and Setup

Load the target image, and test some triangles at different colour values

In [ ]:
using Revise
using Paint
using Serialization
using Images, ImageShow
using Plots
using StaticArrays
using ImageFeatures
using IntervalSets
using Combinatorics
using StatsBase
using Zygote

In [ ]:
target = float.(load("../lisa.png"))
hist = Serialization.deserialize("../output/simresult/simlog_100-prims_1000000-batch_10-epoch_100-refine.bin")
nothing

In [ ]:
ffox = float.(load("../firefox.jpg"))
ffox_resize = imresize(ffox, size(target))

In [ ]:
tri1 = Triangle(Pair(0.25, 0.25), Pair(0.5, 0.25), Pair(0.5, 0.5))
tri2 = Triangle(Pair(0.75, 0.75), Pair(0.5, 0.75), Pair(0.5, 0.5))
nothing

In [ ]:
# newimg = copy(hist.history[end].current)
newimg = copy(ffox_resize)#ones(RGB{Float32}, size(target))
# draw!(newimg, tri1, RGBA{Float32}(0.8508511f0,0.7766122f0,0.6555407f0, 0.5), RasterAlgorithmBounded())
draw!(newimg, tri1, RGBA{Float32}(0.7276809f0,0.5967741f0,0.31770712f0, 0.75), RasterAlgorithmBounded())
# draw!(newimg, tri1, RGBA{Float32}(0.70170283f0,0.5532264f0,0.3110801f0, 0.5), RasterAlgorithmBounded())
println(imloss(target, newimg, SELoss()))
newimg

In [ ]:
# newimg = copy(hist.history[end].current)
newimg = copy(ffox_resize)#ones(RGB{Float32}, size(target))
xs = collect(range(-2, 2, length=1000000))
rlosses = [drawloss(target, newimg, tri1, RGBA{Float32}(r, 0, 0, 0.75), SELoss(), RasterAlgorithmBounded()) for r in xs]
bestr = xs[last(findmin(rlosses))]
glosses = [drawloss(target, newimg, tri1, RGBA{Float32}(0, g, 0, 0.75), SELoss(), RasterAlgorithmBounded()) for g in xs]
bestg = xs[last(findmin(glosses))]
blosses = [drawloss(target, newimg, tri1, RGBA{Float32}(0, 0, b, 0.75), SELoss(), RasterAlgorithmBounded()) for b in xs]
blosses2 = [drawloss(target, newimg, tri1, RGBA{Float32}(0, 0, b, 1.0), SELoss(), RasterAlgorithmBounded()) for b in xs]
bestb = xs[last(findmin(blosses))]
bestr, bestg, bestb

In [ ]:
plotly()
plot(xs, [rlosses, glosses, blosses])

In [ ]:
bestb2 = xs[last(findmin(blosses2))]
bestb, bestb2

In [ ]:
xs[last(findmin(lossmap))]

In [ ]:
(0.70170283f0,0.5532264f0,0.3110801f0) ./ (0.7421574215742157, 0.5683056830568306, 0.3180731807318073)

In [ ]:
blosses

In [ ]:
newimg = copy(hist.history[end].current)
blosses2 = [drawloss(target, newimg, tri1, RGBA{Float32}(1, 0, b, 1.0), AELoss(), RasterAlgorithmBounded()) for b in xs]
minval, minidx = findmin(blosses2)
minval, minidx, xs[minidx]

In [ ]:
newimg = copy(hist.history[end].current)
col = averagepixel(target, tri1, RasterAlgorithmBounded())
drawloss(target, newimg, tri1, RGB{Float32}(1, 0, col.b), AELoss(), RasterAlgorithmBounded())

In [ ]:
import Paint.Raster2D: RasterState, rasterize

struct PA <: RasterState
    current::Array{RGB{Float32}, 2}
    colour::RGB{Float32}
    alpha::Float32
    count::Int32
end

function Paint.Raster2D.rasterfunc(i, j, image, state::PA)
    # @inbounds PA(state.current, state.colour + ((image[i, j] - state.current[i, j]) * state.alpha + state.current[i, j]), state.alpha, state.count + 1)
    @inbounds PA(state.current, state.colour + ((image[i, j] - ((1.0f0 - state.alpha) .* state.current[i, j])) ./ state.alpha), state.alpha, state.count + 1)
end

function PAP(target, current, alpha, shape, algorithm = RasterAlgorithmPointwise())
    state = PA(current, zero(RGB{Float32}), alpha, 0)
    state = rasterize(target, shape, state, algorithm)
    state.colour / Float32(max(Int32(1), state.count))
end

In [ ]:
newimg = copy(ffox_resize)#ones(RGB{Float32}, size(target))
newcol = PAP(target, newimg, 0.75f0, tri1, RasterAlgorithmBounded())
println(newcol)

In [ ]:
2 .* (0.70170283f0,0.5532264f0,0.31108013f0) .- 1

In [ ]:
# testimage = RGB{Float32}.(Gray.(stack([collect(range(0, 1, length=200)) for row in 1:200])))
newimg = ones(RGB{Float32}, size(target))
x = PAP(newimg, newimg, 1.0f0, tri1, RasterAlgorithmBounded())
avg = PAP(target, newimg, 1.0f0, tri1, RasterAlgorithmBounded())
col = clamp01.((avg .- (0.5f0 .* x)) ./ (1.0f0 - 0.5f0))
colalpha = RGBA{Float32}(col.r, col.g, col.b, 0.5f0)

In [ ]:
println(x)
println(avg)
println(col)
RGBA{Float32}(col.r, col.g, col.b, 0.5f0)

In [ ]:
newimg = ones(RGB{Float32}, size(target))
draw!(newimg, tri1, colalpha, RasterAlgorithmBounded())
println(imloss(target, newimg, SELoss()))
newimg

In [ ]:
testimage = RGB{Float32}.(Gray.(stack([collect(range(0, 1, length=200)) for row in 1:200])))
draw!(testimage, tri1, RGBA{Float32}(col.r, col.g, col.b, 0.5f0), RasterAlgorithmBounded())
testimage

In [ ]:
tri1 = Triangle(Pair(0.25, 0.25), Pair(0.5, 0.25), Pair(0.5, 0.5))


In [ ]:
newimg

In [ ]:
tris = hist.history[end].shapes[1:100]
# cols = map(pix -> RGBA{Float32}(pix.r, pix.g, pix.b, 0.75), hist.history[end].current_colours[1:3])
cols = hist.history[end].current_colours[1:100]
background = hist.history[end].background

In [ ]:
function makeloss(colours)
    # newimg = zeros(RGB{Float32}, size(target)) .+ background
    # for k in eachindex(tris)
    #     draw!(newimg, tris[k], colours[k], RasterAlgorithmBounded())
    # end
    # imloss(target, newimg, SELoss())

    w, h = size(target)
    losstotal = 0.0f0

    for y in 1:200
        for x in 1:200
            pix = copy(background)
            for k in eachindex(tris)
                if covers(tris[k], Point(x2u(x, w), y2v(y, h)))
                    pix = over(RGBA{Float32}(colours[k].r, colours[k].g, colours[k].b, 0.75), pix)
                end
            end
            losstotal += loss(pix, target[x, y], SELoss())
        end
    end

    losstotal
end

makeloss(cols)

In [ ]:
cols[1] .+ (0.01f0 .* grads[1])